## General Information & Flowcharts

### Komponenten und Tools

Komponenten RAG Pipe: 
- Parsing 
- Chunking
    - Hugging Face: [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)
    - all-MiniLM-L6-v2
- Embedding
    - nomic-embed-text:latest
    - Ollama show nomic-embed-text:latest 
        - embedding length/dimensions 768 
- Indexing
    - Milvus Vektor DB 
        - Only Ubuntu and macOS -> WSL on Windows mandatory
- Retrieval Augmented Generation 
    - Generation Model: OpenAIChatCompletionsModel (ollama3.2:latest)

Tools:
- diet_comparison (RAG)
- bmi_calculator 
- calorie_calculator
- allergy_check (RAG) 

### Workflow
![Flowchart Workflow](assets/Bild1.png)

### RAG Process
![Floachart RAG](assets/Bild2.png)

## Setup

### 🛠️ Imports

In [1]:
from __future__ import annotations
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List, Literal, pprint, glob, asyncio, datetime, date, time, timezone, ZoneInfo, uuid, dataclass
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, DDGS, rprint
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import Markdown, HTML, JSON, display, widgets
from aiworkshop_utils.openai_imports import OpenAI, Agent, Runner, InputGuardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, ModelSettings, function_tool, trace, ResponseContentPartDoneEvent, ResponseTextDeltaEvent, RawResponsesStreamEvent, TResponseInputItem, ItemHelpers, MessageOutputItem, RunContextWrapper, input_guardrail, output_guardrail
from aiworkshop_utils import config

In [2]:
# Standard library imports
import os
import json
import base64
import logging
from typing import Optional, List, Dict, Any, Union, Literal
import asyncio
from datetime import datetime, date, time, timezone
from zoneinfo import ZoneInfo
import uuid
from dataclasses import dataclass
from pprint import pprint
from glob import glob

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import requests
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from rich import print as rprint
from tqdm import tqdm
from pymilvus import DataType, MilvusClient, Collection
from duckduckgo_search import DDGS

# Document processing imports
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.chunking import HybridChunker
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions,
    PdfPipelineOptions,
)
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline

# OpenAI and Agent imports
from openai import OpenAI, AsyncOpenAI
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrail,
    InputGuardrailTripwireTriggered,
    ModelSettings,
    OpenAIChatCompletionsModel,
    Runner,
    function_tool,
    set_tracing_disabled,
    trace,
    RawResponsesStreamEvent,
    ItemHelpers, 
    MessageOutputItem,
    RunContextWrapper,
    input_guardrail,
    output_guardrail
)

### Pydantic Model

In [3]:
class HealthData(BaseModel):
    age: int = Field(description="Age in years")
    gender: str = Field(description="e.g. male, female, diverse")
    weight_kg: float = Field(description="Weight in kilograms [kg]")   
    height_cm: float = Field(description="Height in centimeters [cm]")
    allergies: Optional[str] = Field(description="e.g. nuts, gluten")
    eating_habits: str = Field(description="e.g. vegetarian, vegan, omnivore, paleo")
    goal: str = Field(description="e.g. weight loss, muscle gain, maintenance")
    activity_level: str = Field(description="e.g. sedentary, lightly active, moderately active, very active")
    timeCooking: int = Field(description="Time spent cooking per day in minutes")
    healthCondition: Optional[str] = Field(description="e.g. diabetes, hypertension")

## Document

### Parsing

In [4]:
class DocumentParser:
    def __init__(self, converter):
        self.converter = converter

    def parse(self, file_path: str, options: dict = None):
        print(f"Converting document: {file_path}")
        result = self.converter.convert(file_path)
        return result.document

In [5]:
source = "assets/health_data.pdf"

my_processor = DocumentParser(DocumentConverter())
processor_result = my_processor.parse(source)

Converting document: assets/health_data.pdf


### Chunking

In [6]:
class DocumentChunker:
    def __init__(self, tokenizer: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.tokenizer = tokenizer
        self.chunker = HybridChunker(tokenizer=tokenizer)

    def chunk(self, document, options: dict = None):
        print("Chunking document...")
        chunks = list(self.chunker.chunk(document))
        print(f"Created {len(chunks)} chunks")
        return chunks

In [7]:
my_chunker = DocumentChunker()
chunker_result = my_chunker.chunk(processor_result)

Chunking document...
Created 98 chunks


### Embedding

In [8]:
class DocumentEmbedder:
    def __init__(self, url, model_name):
        self.url = url
        self.model_name = model_name

    def _post_request(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        response = requests.post(
            self.url,
            json={
                "model": self.model_name,
                "input": texts
            }
        )
        response.raise_for_status()
        return response.json()

    def get_embeddings(self, texts):
        response = self._post_request(texts)
        embeddings = response["embeddings"]
        if isinstance(texts, str) or len(embeddings) == 1:
            return embeddings[0]
        return embeddings

    def get_full_response(self, texts):
        return self._post_request(texts)

    def embed(self, chunks):
        texts = [chunk.text for chunk in chunks]
        return self.get_embeddings(texts)
    
    def get_prepared_data_for_indexing(self, chunks):
        embedding_result = self.embed(chunks)
        data = []
        for chunk, vector in zip(chunks, embedding_result):
            headings = ""
            page_info = ""
            if hasattr(chunk, "meta") and chunk.meta:
                headings_list = getattr(chunk.meta, "headings", [])
                if headings_list:
                    headings = " > ".join(headings_list)
                page_info = getattr(chunk.meta, "page_info", "")
            data.append({
                "vector": vector,
                "text": chunk.text,
                "headings": headings,
                "page_info": page_info
            })
        return data

In [9]:
my_embedder = DocumentEmbedder(url=config.OAPI_EMBED_URL, model_name=config.OMODEL_NOMIC)
embedding_result = my_embedder.embed(chunker_result)

In [10]:
print(len(embedding_result))
print(embedding_result[0])

98
[0.029407937, -0.007725332, -0.16320756, -0.026395265, 0.03437459, -0.03793293, -0.049199972, -0.0077174124, -0.021334266, -0.021033404, -0.03016901, -8.664924e-05, 0.07509571, 0.085213125, 0.008133813, -0.010943744, -0.016439993, -0.043249737, -0.024111161, 0.010377481, 0.05121823, 0.03195553, -0.009324543, -0.0596079, 0.025305636, 0.0256475, -0.0048430045, 0.002121591, -0.04049873, -0.0050582634, 0.0074854787, 0.00644616, 0.0057679694, -0.00082253775, 7.414384e-05, -0.075304456, 0.016309405, 0.005631348, -0.024207316, 0.015241426, 0.02103698, -0.028154803, 0.001779003, -0.060931705, -0.008009651, 0.0022962943, -0.04469948, -0.0023547928, 0.07105846, -0.06433149, 0.0021380114, 0.043761663, -0.015382296, 0.020869996, 0.078693844, -0.036903802, -0.07706409, -0.05066658, -0.013687871, -0.053277984, 0.021080783, 0.078174956, -0.065949656, 0.014262262, 0.034166154, -0.026743347, -0.06996673, 0.056965724, 0.008793368, 0.0023380641, 0.0127667915, -0.009422646, 0.035293605, 0.0011332023, 0

In [11]:
prepared_embeddings = my_embedder.get_prepared_data_for_indexing(chunker_result)
print(prepared_embeddings[0])

{'vector': [0.029407937, -0.007725332, -0.16320756, -0.026395265, 0.03437459, -0.03793293, -0.049199972, -0.0077174124, -0.021334266, -0.021033404, -0.03016901, -8.664924e-05, 0.07509571, 0.085213125, 0.008133813, -0.010943744, -0.016439993, -0.043249737, -0.024111161, 0.010377481, 0.05121823, 0.03195553, -0.009324543, -0.0596079, 0.025305636, 0.0256475, -0.0048430045, 0.002121591, -0.04049873, -0.0050582634, 0.0074854787, 0.00644616, 0.0057679694, -0.00082253775, 7.414384e-05, -0.075304456, 0.016309405, 0.005631348, -0.024207316, 0.015241426, 0.02103698, -0.028154803, 0.001779003, -0.060931705, -0.008009651, 0.0022962943, -0.04469948, -0.0023547928, 0.07105846, -0.06433149, 0.0021380114, 0.043761663, -0.015382296, 0.020869996, 0.078693844, -0.036903802, -0.07706409, -0.05066658, -0.013687871, -0.053277984, 0.021080783, 0.078174956, -0.065949656, 0.014262262, 0.034166154, -0.026743347, -0.06996673, 0.056965724, 0.008793368, 0.0023380641, 0.0127667915, -0.009422646, 0.035293605, 0.00113

### Vektor-DB

In [12]:
class VectorDBCreator:
    def __init__(self, milvus_client_name):
        self.milvus_client = MilvusClient(f"{milvus_client_name}.db")

    def create_collection(self, collection_name: str, dimension: int, **kwargs):
        if self.milvus_client.has_collection(collection_name=collection_name):
            self.milvus_client.drop_collection(collection_name=collection_name)
        self.milvus_client.create_collection(
            collection_name=collection_name,
            dimension=dimension,
            primary_field_name='id',
            id_type=DataType.INT64,
            vector_field_name='vector',
            extra_fields=[
                {"name": "text", "type": DataType.VARCHAR, "max_length": 1024},
                {"name": "headings", "type": DataType.VARCHAR, "max_length": 512},
                {"name": "page_info", "type": DataType.VARCHAR, "max_length": 128}
            ],
            metric_type='IP',
            auto_id=True,
            consistency_level='Strong',
            **kwargs
        )
        print(f"Collection '{collection_name}' with dimension {dimension} created.")
    
    def get_milvus_client(self):
        return self.milvus_client

In [13]:
my_vdb_creator = VectorDBCreator("my_vector_db_01")
my_vdb_creator.create_collection("my_collection", dimension=768)

Collection 'my_collection' with dimension 768 created.


### Indexing

In [14]:
class DocumentIndexer:
    def __init__(self, milvus_client, collection_name: str):
        self.milvus_client = milvus_client
        self.collection_name = collection_name

    def index(self, data: list) -> dict:
        print(f"Inserting {len(data)} vectors into collection '{self.collection_name}'...")
        self.milvus_client.insert(collection_name=self.collection_name, data=data)
        stats = {"indexed_count": len(data)}
        print(f"Finished indexing: {stats['indexed_count']} vectors inserted.")
        return stats

In [15]:
my_document_indexer = DocumentIndexer(my_vdb_creator.get_milvus_client(), "my_collection")
my_document_indexer.index(prepared_embeddings)

Inserting 98 vectors into collection 'my_collection'...
Finished indexing: 98 vectors inserted.


{'indexed_count': 98}

### Retrieval

In [16]:
class DocumentRetriever:
    def __init__(self, milvus_client, collection_name: str, embedder):
        self.milvus_client = milvus_client
        self.collection_name = collection_name
        self.embedder = embedder

    def retrieve(self, query: str, k: int = 5) -> list:
        
        print(f"Processing query: {query}")
        query_embedding = self.embedder.get_embeddings(query)
        search_results = self.milvus_client.search(
            collection_name=self.collection_name,
            data=[query_embedding],
            limit=k,
            search_params={"metric_type": "IP", "params": {}},
            output_fields=["text", "headings", "page_info"]
        )
        results = []
        for res in search_results[0]:
            entity = res["entity"]
            results.append({
                "text": entity.get("text", ""),
                "headings": entity.get("headings", []),
                "page_info": entity.get("page_info", None),
                "distance": res["distance"]
            })
        return results

    def format_context(self, chunks: list) -> str:
        context_parts = []
        for i, chunk in enumerate(chunks):
            headings_field = chunk.get("headings", None)
            if isinstance(headings_field, list):
                heading_path = " > ".join(headings_field) if headings_field else "Document section"
            elif isinstance(headings_field, str):
                heading_path = headings_field or "Document section"
            else:
                heading_path = "Document section"
                
            page_ref = f"(Page {chunk.get('page_info')})" if chunk.get('page_info') else ""
            context_parts.append(
                f"EXCERPT {i+1} - {heading_path} {page_ref}:\n{chunk['text']}\n"
            )
        return "\n".join(context_parts)

In [17]:
my_document_retriever = DocumentRetriever(my_vdb_creator.get_milvus_client(), "my_collection", my_embedder)

retriever_results = my_document_retriever.retrieve("Wie viele Kalorien brauche ich bei wenig Bewegung?", k=3)
show_pretty_json(retriever_results)

Processing query: Wie viele Kalorien brauche ich bei wenig Bewegung?


```json
[
  {
    "text": "Um deinen Kalorienbedarf zu berechnen, musst du deinen Grundumsatz (BMR) und deinen Aktivit\u00e4tsfaktor kennen. Der BMR gibt an, wie viele Kalorien du im Ruhezustand verbrauchst, und der Aktivit\u00e4tsfaktor ber\u00fccksichtigt deine t\u00e4glichen Aktivit\u00e4ten.",
    "headings": "8.3 Wie berechne ich meinen Kalorienbedarf?",
    "page_info": "",
    "distance": 0.6846004724502563
  },
  {
    "text": "Der Kalorienbedarf eines Menschen h\u00e4ngt von verschiedenen Faktoren ab, wie Alter, Geschlecht, Gewicht, Gr\u00f6\u00dfe und Aktivit\u00e4tsniveau. Der Grundumsatz (BMR) ist die Anzahl der Kalorien, die der K\u00f6rper in Ruhe ben\u00f6tigt, um lebenswichtige Funktionen aufrechtzuerhalten. Der Gesamtenergieumsatz (TDEE) ber\u00fccksichtigt zus\u00e4tzlich die k\u00f6rperliche Aktivit\u00e4t.",
    "headings": "5.1 Kalorienbedarf",
    "page_info": "",
    "distance": 0.6833980083465576
  },
  {
    "text": "- \u00b7 Kalorien insgesamt: 1.700 kcal",
    "headings": "7.1.5 Gesamtkalorien und Makron\u00e4hrstoffe",
    "page_info": "",
    "distance": 0.6613255739212036
  }
]
```

In [18]:
formatted_context = my_document_retriever.format_context(retriever_results)
rprint(formatted_context)

EXCERPT 1 - 8.3 Wie berechne ich meinen Kalorienbedarf? :
Um deinen Kalorienbedarf zu berechnen, musst du deinen Grundumsatz (BMR) und deinen Aktivitätsfaktor kennen. Der 
BMR gibt an, wie viele Kalorien du im Ruhezustand verbrauchst, und der Aktivitätsfaktor berücksichtigt deine 
täglichen Aktivitäten.

EXCERPT 2 - 5.1 Kalorienbedarf :
Der Kalorienbedarf eines Menschen hängt von verschiedenen Faktoren ab, wie Alter, Geschlecht, Gewicht, Größe und 
Aktivitätsniveau. Der Grundumsatz (BMR) ist die Anzahl der Kalorien, die der Körper in Ruhe benötigt, um 
lebenswichtige Funktionen aufrechtzuerhalten. Der Gesamtenergieumsatz (TDEE) berücksichtigt zusätzlich die 
körperliche Aktivität.

EXCERPT 3 - 7.1.5 Gesamtkalorien und Makronährstoffe :
- · Kalorien insgesamt: 1.700 kcal

## Agent

### Agent-Tool für RAG

In [19]:
# Configuration
class Config:
    OLLAPI_ENDPOINT_BASE = 'http://localhost:11434/v1'  # Base endpoint for Ollama
    OMODEL_LLAMA3D2 = 'llama3.2:latest'  # Model name

# Context class for agent state / Configurations for agent
class RAGContext(BaseModel):
    question: str = ""
    formatted_context: str = ""
    language: str = "English"  # Default language for responses

# Initialize the model - separate function for clarity
def create_llm_model():
    # Disable tracing to avoid messages about missing API keys
    set_tracing_disabled(True)
    
    # Create model with your endpoint
    return OpenAIChatCompletionsModel(
        model=config.OMODEL_LLAMA3D2,
        openai_client=AsyncOpenAI(
            base_url=config.OLLAPI_ENDPOINT_BASE, 
            api_key="fake-key"  # Using fake key as local endpoint doesn't require auth
        )
    )

# Helper Function to extract information from prompt
async def extract_health_info(prompt: str) -> HealthData:
    data = {
        "model": config.OMODEL_LLAMA3D2,
        "prompt": prompt + "\nAnswer in JSON, based on the HealthData-Class.",
        "stream": False,
        "format": HealthData.model_json_schema()
    }
    response = requests.post(config.OAPI_GENERATE_URL, json=data)
    response_json = response.json()
    response_data = json.loads(response_json["response"])
    return HealthData(**response_data)

# Diet comparison tool that uses DocumentRetriever
@function_tool
async def diet_comparison(
    context: RunContextWrapper[RAGContext], 
    query: str, 
    k: int = 4
    ) -> str:
    """
    Compares retrieved information about various diets.
    """

    try:
        # Store the question in context
        context.context.question = query
        
        # Use your existing document retriever
        # retriever_results = my_document_retriever.retrieve(query, k=k)
        retriever_results = my_document_retriever.retrieve(
            f"{query} Compare different diets in terms of nutrition, benefits, and drawbacks.", 
            k=k
        )
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Store formatted context in the agent context
        context.context.formatted_context = formatted_context
        
        return formatted_context
    except Exception as e:
        error_msg = f"Error retrieving documents: {str(e)}"
        print(error_msg)
        return error_msg
    
# BMI calculator
@function_tool
def bmi_calculator(height_m, weight_kg) -> dict:
    """
    Calculates the Body Mass Index (BMI) with given parameters.

    :param height_m: height in meters
    :param weight_kg: weight in kilograms 
    :return: BMI
    """
    
    response = weight_kg / (height_m * height_m)
    return {
        "BMI": round(response, 2)
    }
    
# Calorie calculator
@function_tool
async def calorie_calculator(gender: str, weight_kg: float, height_cm: float, age: int, activity_level: str) -> dict:
    """
    Calculates Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure (TDEE)
    using the Mifflin-St. Jeor equation.

    :param gender: "m" for male or "f" for female
    :param weight_kg: Weight in kilograms
    :param height_cm: Height in centimeters
    :param age: Age in years
    :param activity_level: One of the keys from `activity_factors`
    :return: Dictionary with BMR, TDEE, and activity factor used
    """
    activity_factors = {
        "sedentary": 1.2,        # little or no exercise
        "light": 1.375,          # light exercise 1–3 days/week
        "moderate": 1.55,        # moderate exercise 3–5 days/week
        "active": 1.725,         # hard exercise 6–7 days/week
        "very active": 1.9       # very intense physical job or training
    }
    
    gender = gender.lower()
    if gender in ["m","male","man"]:
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    elif gender in ["f","female","woman"]:
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161
    else:
        raise ValueError("Invalid gender. Use 'm' for male or 'f' for female.")

    factor = activity_factors.get(normalize_activity_level(activity_level))
    if factor is None:
        raise ValueError(f"Invalid activity level: {activity_level}")

    tdee = bmr * factor

    return {
        "BMR": round(bmr, 2),
        "TDEE": round(tdee, 2),
        "Activity Factor": factor
    }

# Helper function for calorie calculator
def normalize_activity_level(input_level):
    input_level = input_level.lower()
    if "sedentary" in input_level:
        return "sedentary"
    elif "light" in input_level:
        return "light"
    elif "moderate" in input_level:
        return "moderate"
    elif "active" in input_level and "very" not in input_level:
        return "active"
    elif "very active" in input_level or ("very" in input_level and "active" in input_level):
        return "very active"
    else:
        raise ValueError(f"Unrecognized activity level input: {input_level}")

# Allergy information tool
@function_tool
async def allergy_check(
    context: RunContextWrapper[RAGContext], 
    query: str, 
    k: int = 4
    ) -> str:
    """
    Retrieves information about potential allergies in diets.
    """

    try:
        # Store the question in context
        context.context.question = query
        
        # Use your existing document retriever
        # retriever_results = my_document_retriever.retrieve(query, k=k)
        retriever_results = my_document_retriever.retrieve(
            f"{query} Find information about allergens, intolerances, or dietary restrictions.", 
            k=k
        )
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Store formatted context in the agent context
        context.context.formatted_context = formatted_context
        
        return formatted_context
    except Exception as e:
        error_msg = f"Error retrieving documents: {str(e)}"
        print(error_msg)
        return error_msg

# Create a single RAG agent - simplifying the design
# Using valid tool_use_behavior value
rag_agent = Agent[RAGContext](
    name="Food Advisory Assistant",
    instructions="""
    You are an assistant specialized in dietary advice and health information.
    
    WORKFLOW:
    1. When a user asks a question regarding diets, use the diet_comparison tool to retrieve relevant information
    2. When a user asks a question regarding BMI, use the bmi_calculator tool to retrieve relevant information
    3. When a user asks a question regarding calories, use the calorie_calculator tool to retrieve relevant information
    4. When a user asks a question regarding allergies, use the allergy_check tool to retrieve relevant information
    5. Carefully analyze the retrieved context
    6. Provide a clear, accurate answer based on the retrieved information
    7. If the information isn't available in the retrieved context, indicate this clearly
    
    Respond in English.
    Be helpful, accurate, and concise in your responses.
    """,
    tools=[diet_comparison, bmi_calculator, allergy_check, calorie_calculator],
    model=create_llm_model(),
    # Using a valid tool_use_behavior value
    tool_use_behavior="run_llm_again",
    # Set model settings to help with function calling
    model_settings=ModelSettings(
        temperature=0.1,  # Lower temperature for more deterministic responses
        tool_choice="auto"  # Auto tool choice
    )
)

# Manual process for RAG when the model doesn't handle function calling properly
async def manual_rag_process(question, k=1):
    """
    Manually execute the RAG process when the model doesn't properly use function calling.
    """
    try:
        # Direct call to retrieve documents
        retriever_results = my_document_retriever.retrieve(question, k=k)
        formatted_context = my_document_retriever.format_context(retriever_results)
        
        # Create a prompt with the retrieved context
        formatted_prompt = f"""
        Question: {question}

        Context from diet plans:
        {formatted_context}

        Based on the above context, please provide a concise and accurate answer to the question.
        """
        
        # Create context with the retrieved info
        context = RAGContext(
            question=question,
            formatted_context=formatted_context,
            language="English"
        )
        
        # Use a list for input items with the formatted prompt
        input_items = [{"content": formatted_prompt, "role": "user"}]
        
        # Run the model with this prompt
        run_result = await Runner.run(
            rag_agent,
            input=input_items,
            context=context
        )
        
        # Return the results
        return {
            "answer": run_result.final_output,
            "run_result": run_result,
            "context": context
        }
    except Exception as e:
        error_msg = f"Error in manual RAG process: {str(e)}"
        print(error_msg)
        return {"error": error_msg}

# Detailed trace function for better visibility into the agent process
def print_run_trace(run_result):
    print("\n== DETAILED AGENT RUN TRACE ==\n")
    
    # Print basic info
    print(f"Total items generated: {len(run_result.new_items)}")
    print(f"Model responses: {len(run_result.raw_responses)}")
    print("-" * 80)
    
    # Loop through each item and print details based on item type
    for i, item in enumerate(run_result.new_items):
        item_type = getattr(item, 'type', 'unknown')
        
        print(f"\n[STEP {i+1}: {item_type}]")
        
        if item_type == 'tool_call_item':
            # Print tool call details
            print(f"  Agent: {item.agent.name}")
            raw_item = item.raw_item
            print(f"  Tool called: {raw_item.name}")
            print(f"  Arguments: {raw_item.arguments}")
            
        elif item_type == 'tool_call_output_item':
            # Print tool output details
            print(f"  Agent: {item.agent.name}")
            print(f"  Output type: {item.raw_item.get('type', 'unknown')}")
            
            # Truncate long outputs for readability
            output = item.output
            if len(output) > 150:
                output = output[:150] + "..."
            print(f"  Output: {output}")
            
        elif item_type == 'message_output_item':
            # Print message details
            print(f"  Agent: {item.agent.name}")
            raw_item = item.raw_item
            
            # Extract and format content
            content = ""
            if hasattr(raw_item, 'content') and raw_item.content:
                for content_item in raw_item.content:
                    if hasattr(content_item, 'text'):
                        text = content_item.text
                        if len(text) > 150:
                            text = text[:150] + "..."
                        content = text
            
            print(f"  Role: {getattr(raw_item, 'role', 'unknown')}")
            print(f"  Content: {content}")
            
        else:
            # For any other item types
            print(f"  Item details: {item}")
            
        print("-" * 80)
    
    # Print token usage information if available
    print("\n== TOKEN USAGE ==")
    total_input_tokens = 0
    total_output_tokens = 0
    
    for i, response in enumerate(run_result.raw_responses):
        if hasattr(response, 'usage'):
            usage = response.usage
            input_tokens = getattr(usage, 'input_tokens', 0)
            output_tokens = getattr(usage, 'output_tokens', 0)
            total_tokens = getattr(usage, 'total_tokens', 0)
            
            print(f"Response {i+1}:")
            print(f"  Input tokens: {input_tokens}")
            print(f"  Output tokens: {output_tokens}")
            print(f"  Total tokens: {total_tokens}")
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
    
    print(f"\nTotal input tokens: {total_input_tokens}")
    print(f"Total output tokens: {total_output_tokens}")
    print(f"Grand total tokens: {total_input_tokens + total_output_tokens}")

# Main function with detection of function calling issues
async def ask(
    question: str, 
    health_data : str,
    document_retriever=None, 
    language="English"
    ) -> Dict:
    # Allow passing a document retriever if not defined globally
    global my_document_retriever
    if document_retriever is not None:
        my_document_retriever = document_retriever
    
    # Create context with specified language
    context = RAGContext(language=language)
    
    # First attempt: standard approach
    with trace("Food Advisory Assistant - Standard Approach"):
        # Format health_data
        formatted_health_data = await extract_health_info(health_data)
        print("----- Extracted Data -----")
        print(formatted_health_data.model_dump_json(indent=4))

        # Format question
        formatted_question = f"{question} My Health Data: {formatted_health_data.model_dump_json()}"
        print(f"\nYOUR QUESTION:")
        print(f"\n{formatted_question}\n")

        # Use a list for input items
        input_items = [{"content": formatted_question, "role": "user"}]
        
        # Run the agent
        run_result = await Runner.run(
            rag_agent,
            input=input_items,
            context=context
        )
    
    # Check if we got a proper answer or just a function call spec
    is_function_call_text = False
    if run_result.final_output:
        # Check if the output looks like a raw function call
        if run_result.final_output.startswith('{"name":') or \
           'diet_comparison' in run_result.final_output:
            is_function_call_text = True
    
    # If the model returned a function call as text, use manual RAG approach
    if is_function_call_text:
        print("Detected function call specification in output. Switching to manual RAG process...")
        return await manual_rag_process(str(formatted_question), k=1)
    
    # Standard approach worked fine
    return {
        "answer": run_result.final_output,
        "run_result": run_result,
        "context": context
    }

# Example usage
async def demo_rag(question: str, health_data: str):
    
    result = await ask(question, health_data, my_document_retriever)
    
    # Print the answer
    print("ANSWER:")
    print(result["answer"])
    
    # Use our detailed trace function
    print_run_trace(result['run_result'])
    
    # Print context preview
    context = result.get('context')
    if context and hasattr(context, 'formatted_context'):
        print("\n== RETRIEVED CONTEXT ==")
        print(context.formatted_context)
    
    return 'success' #result

## Examples

### Prompt 1

In [20]:
# For Jupyter notebook execution
health_data1: str = """
    I am 20 years old, male, weigh 85kg, and am 180 cm tall.
    I am lactose intolerant and enjoy eating vegan.
    My goal is to build more muscle, which is why I exercise moderately.
    For daily cooking, I plan to spend a maximum of 2 hours (120 minutes).
    Unfortunately, I am also diabetic, so I need to keep an eye on my blood sugar levels.
    """

question1: str = "What is my BMI?"

await demo_rag(question1, health_data1)

----- Extracted Data -----
{
    "age": 20,
    "gender": "male",
    "weight_kg": 85.0,
    "height_cm": 180.0,
    "allergies": "lactose intolerant",
    "eating_habits": "vegan",
    "goal": "build muscle",
    "activity_level": "moderate exercise",
    "timeCooking": 120,
    "healthCondition": "diabetes"
}

YOUR QUESTION:

What is my BMI? My Health Data: {"age":20,"gender":"male","weight_kg":85.0,"height_cm":180.0,"allergies":"lactose intolerant","eating_habits":"vegan","goal":"build muscle","activity_level":"moderate exercise","timeCooking":120,"healthCondition":"diabetes"}

ANSWER:
Based on the information provided, your BMI is approximately 26.23. This falls into the overweight category according to the World Health Organization's classification.

Please note that a BMI of 26.23 may be a concern for someone with diabetes, as it suggests an increased risk of developing insulin resistance and other health complications. However, it's essential to consider individual factors such 

'success'

### Prompt 2

In [21]:
# For Jupyter notebook execution

health_data2: str = """
    I'm a 35-year-old woman, weighing 70kg and 165cm tall.
    I have a gluten allergy and follow a vegetarian diet.
    My primary goal is to lose weight.
    I’m fairly active and work out around 5 times a week.
    I can only spend about 45 minutes cooking each day.
    I also have mild hypertension that I need to manage.
    """

question2: str = "How many calories do I need when I'm moderately active?"

await demo_rag(question2, health_data2)

----- Extracted Data -----
{
    "age": 35,
    "gender": "female",
    "weight_kg": 70.0,
    "height_cm": 165.0,
    "allergies": "gluten",
    "eating_habits": "vegetarian",
    "goal": "lose weight",
    "activity_level": "active",
    "timeCooking": 45,
    "healthCondition": "mild hypertension"
}

YOUR QUESTION:

How many calories do I need when I'm moderately active? My Health Data: {"age":35,"gender":"female","weight_kg":70.0,"height_cm":165.0,"allergies":"gluten","eating_habits":"vegetarian","goal":"lose weight","activity_level":"active","timeCooking":45,"healthCondition":"mild hypertension"}

ANSWER:
Based on your health data and activity level, your daily calorie needs for moderate activity are approximately 2406.81 calories. This is calculated using the Total Daily Energy Expenditure (TDEE) formula, which takes into account your basal metabolic rate (BMR), activity factor, and other factors such as age, gender, weight, and height.

As a moderately active female with a goal 

'success'

### Prompt 3

In [22]:
health_data3: str = """
    I’m 28, male, and weigh 95kg with a height of 185cm.
    No known allergies, and I follow an omnivorous diet.
    My goal is to maintain my current weight while staying fit.
    I lead a sedentary lifestyle due to a desk job, but I do walk occasionally.
    I usually cook for around 30 minutes per day.
    I don’t have any medical conditions.
    """

question3: str = "What kind of diet can you recommend?"

await demo_rag(question3, health_data3)

----- Extracted Data -----
{
    "age": 28,
    "gender": "male",
    "weight_kg": 95.0,
    "height_cm": 185.0,
    "allergies": null,
    "eating_habits": "omnivorous",
    "goal": "maintain current weight and fitness level",
    "activity_level": "sedentary",
    "timeCooking": 30,
    "healthCondition": null
}

YOUR QUESTION:

What kind of diet can you recommend? My Health Data: {"age":28,"gender":"male","weight_kg":95.0,"height_cm":185.0,"allergies":null,"eating_habits":"omnivorous","goal":"maintain current weight and fitness level","activity_level":"sedentary","timeCooking":30,"healthCondition":null}

ANSWER:
Based on the user's health data, I recommend a balanced and varied diet that meets their omnivorous eating habits. Considering their sedentary lifestyle, it's essential to focus on nutrient-dense foods that provide sustained energy.

Here are some general dietary recommendations:

1. **Eat more protein**: As a male, you need about 0.8-1 gram of protein per kilogram of body w

'success'

### Prompt 4

In [23]:
health_data4: str = """
    Female, 23 years old, 60kg, 170cm.
    I’m allergic to nuts and prefer a vegan diet.
    My goal is to build lean muscle.
    I go to the gym 3–4 times a week — moderately active.
    I enjoy cooking and can dedicate up to 90 minutes a day.
    No health issues.
    """

question4: str = "Which diets respect my allergy?"

await demo_rag(question4, health_data4)

----- Extracted Data -----
{
    "age": 23,
    "gender": "female",
    "weight_kg": 60.0,
    "height_cm": 170.0,
    "allergies": null,
    "eating_habits": ",vegan",
    "goal": ",lean muscle building",
    "activity_level": ",moderately active",
    "timeCooking": 90,
    "healthCondition": null
}

YOUR QUESTION:

Which diets respect my allergy? My Health Data: {"age":23,"gender":"female","weight_kg":60.0,"height_cm":170.0,"allergies":null,"eating_habits":",vegan","goal":",lean muscle building","activity_level":",moderately active","timeCooking":90,"healthCondition":null}

Processing query: diets that respect vegan allergy Compare different diets in terms of nutrition, benefits, and drawbacks.
ANSWER:
Es scheint, als ob die vorliegende Textsammlung eine umfassende Übersicht über verschiedene Aspekte der Ernährung und Gesundheit bietet. Sie enthält Informationen zu verschiedenen Diätformen, Makronährstoffverteilungen, Lebensmittelquellen und -empfehlungen sowie Rezepte und Snacks.



'success'

### Prompt 5

In [24]:
health_data5: str = """
    I'm 50 years old, non-binary, 78kg, and 172cm tall.
    I eat paleo and don’t have any allergies.
    My main goal is to reduce body fat and improve energy levels.
    I’m lightly active — I walk daily and occasionally cycle.
    I cook around 60 minutes each day.
    I have pre-diabetes and want to manage it through diet.
    """

question5: str = "What is my BMI and which diet should I aim for?"

await demo_rag(question5, health_data5)

----- Extracted Data -----
{
    "age": 50,
    "gender": "non-binary",
    "weight_kg": 78.0,
    "height_cm": 172.0,
    "allergies": null,
    "eating_habits": "paleo",
    "goal": "reduce body fat and improve energy levels",
    "activity_level": "lightly active",
    "timeCooking": 60,
    "healthCondition": "pre-diabetes"
}

YOUR QUESTION:

What is my BMI and which diet should I aim for? My Health Data: {"age":50,"gender":"non-binary","weight_kg":78.0,"height_cm":172.0,"allergies":null,"eating_habits":"paleo","goal":"reduce body fat and improve energy levels","activity_level":"lightly active","timeCooking":60,"healthCondition":"pre-diabetes"}



2025-04-15 17:56:11,564 [ERROR][handler]: RPC error: [search], <ParamError: (code=1, message=`limit` value 0 is illegal)>, <Time:{'RPC start': '2025-04-15 17:56:11.564220', 'RPC error': '2025-04-15 17:56:11.564322'}> (decorators.py:140)
2025-04-15 17:56:11,565 [ERROR][search]: Failed to search collection: my_collection (milvus_client.py:415)


Processing query: non-binary, 50 years old, 78 kg, 172 cm, reduce body fat and improve energy levels, lightly active, paleo diet. Compare different diets in terms of nutrition, benefits, and drawbacks.
Error retrieving documents: <ParamError: (code=1, message=`limit` value 0 is illegal)>
Detected function call specification in output. Switching to manual RAG process...
Processing query: What is my BMI and which diet should I aim for? My Health Data: {"age":50,"gender":"non-binary","weight_kg":78.0,"height_cm":172.0,"allergies":null,"eating_habits":"paleo","goal":"reduce body fat and improve energy levels","activity_level":"lightly active","timeCooking":60,"healthCondition":"pre-diabetes"}
Processing query: What diet is suitable for someone with pre-diabetes, who is non-binary, 50 years old, 78 kg, 172 cm tall and has a paleo eating habit? Compare different diets in terms of nutrition, benefits, and drawbacks.
ANSWER:
Based on the context provided, I was unable to determine your BMI. Ho

'success'

### Prompt 6

In [25]:
health_data6: str = """
    I’m a 42-year-old man, 88kg, 178cm.
    No food allergies, and I eat everything — classic omnivore.
    I’d like to gain muscle mass, especially for strength training.
    I’m very active, training intensely 6 days a week.
    I cook every day for about 75 minutes.
    No specific health issues.
    """

question6: str = "Which vitamins do I need and how do I gain them?"

await demo_rag(question6, health_data6)

----- Extracted Data -----
{
    "age": 42,
    "gender": "male",
    "weight_kg": 88.0,
    "height_cm": 178.0,
    "allergies": null,
    "eating_habits": "omnivore",
    "goal": "muscle_gain",
    "activity_level": "high",
    "timeCooking": 75,
    "healthCondition": null
}

YOUR QUESTION:

Which vitamins do I need and how do I gain them? My Health Data: {"age":42,"gender":"male","weight_kg":88.0,"height_cm":178.0,"allergies":null,"eating_habits":"omnivore","goal":"muscle_gain","activity_level":"high","timeCooking":75,"healthCondition":null}

Processing query: vitamins needed for muscle gain in omnivores with high activity level Compare different diets in terms of nutrition, benefits, and drawbacks.
ANSWER:
Es scheint, als ob die vorherigen Antworten nicht direkt mit dem Text zusammenhängen, den du angefordert hast. Da ich jedoch keine spezifischen Anforderungen oder Fragen aus deinem Text erhalten habe, werde ich versuchen, eine allgemeine Antwort zu geben, die auf deine Frage "Wa

'success'

### Promp 7

In [26]:
health_data7: str = """
    
    """

question7: str = "What is my calorie use and which diet should I consider?"

await demo_rag(question7, health_data7)

----- Extracted Data -----
{
    "age": 25,
    "gender": "female",
    "weight_kg": 55.0,
    "height_cm": 160.0,
    "allergies": null,
    "eating_habits": "omnivore",
    "goal": "lose_weight",
    "activity_level": "moderate",
    "timeCooking": 60,
    "healthCondition": "hypertension"
}

YOUR QUESTION:

What is my calorie use and which diet should I consider? My Health Data: {"age":25,"gender":"female","weight_kg":55.0,"height_cm":160.0,"allergies":null,"eating_habits":"omnivore","goal":"lose_weight","activity_level":"moderate","timeCooking":60,"healthCondition":"hypertension"}

ANSWER:
Based on the calorie calculator output, your Total Daily Energy Expenditure (TDEE) is approximately 1959.2 calories.

Considering your goal to lose weight and activity level as moderate, a suitable diet for you could be a balanced and sustainable approach such as the Mediterranean Diet or the DASH Diet. Both diets focus on whole, unprocessed foods like fruits, vegetables, whole grains, lean prote

'success'

# Stärken, Schwächen und Schwierigkeiten

## Stärken

- Agent erkennt Kontext und wählt danach Tools aus
- BMI und Calorie Calculator funktionieren einwandfrei
- Findet erfolgreich Referenzen in PDF und generiert daraus Antworten

## Schwächen

- Schwierigkeiten bei sprachlichen Unterschieden (Deutsch -> English / English -> Deutsch)
    - Primäre Sprache für Antworten ist eigentlich Englisch -> antwortet teilweise trotzdem auf Deutsch
    - Prompts und Knowledge-Base sollten in einheitlicher Sprache bleiben
- Daten wie HealthData sind nicht ganz nachvollziehbar. Bspw. obwohl diese nicht angegeben werden werden die Felder trotzdem ausgefüllt (Halluzinationen oder Zwischenspeicher?)
- Gibt manchmal eine falsche Auskunft -> Bspw. Schlägt Mahlzeiten von anderen Diätsformen vor (die in der vorgeschlagenen Diätform nicht vorkommen) 
- Trotz Anweisung gibt es hin und wieder Halluzinationen bei fehlenden Informationen
- Weicht bei manchen Fragen aus, die nicht explizit durch ein Tool beantwortet werden können

## Schwierigkeiten

- Milvus DB Limit=0 Fehler: Error retrieving documents: <ParamError: (code=1, message=`limit` value 0 is illegal)>
    - Hängt vermutlich mit [Retrieval](#Retrieval) und dem Parameter "k" zusammen. 
    - k wird manchmal auf 0 gesetzt, weshalb auch das limit auf 0 gesetzt wird -> Milvus DB kann mit limit=0 nicht umgehen

![image.png](assets/Bild3.png)

- WSL auf Windows:
    - Wird benötigt, weil Milvus DB Windows Systeme nicht unterstützt (bisher nur Ubuntu und macOS - [siehe milvus Repo](https://github.com/milvus-io/milvus-lite?tab=readme-ov-file#requirements) )
    - Da mit WSL ein eigenes OS auf Windows betrieben wird, sollte z.B. das Projekt in der WSL Umgebung aufgesetzt werden (also in keinem Windows Ordner liegen). 
        - Damit sollte auch das Problem mit dem nicht erfassten Kernel gelöst sein.
    - Eventuell muss auch Ollama extra für WSL installiert und ausgeführt werden. (Default-)API-Endpoints müssen entweder anders konfiguriert werden, oder es darf nur Ollama aus der WSL Umgebung aktiv sein (z.B. auf Windows gar nicht installieren und nur mit WSL arbeiten). 